In [1]:
import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import re

In [2]:
def make_soup(url):
#     r = requests.get(url)
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    r = urlopen(req).read()
    soup = BeautifulSoup(r, "html")
    return soup

# put urls in a list
def get_xml_urls(soup):
    urls = [loc.string for loc in soup.find_all("loc")]
    return urls

# get media urls
def get_urls(xml):
    list_of_urls = []
    soup = make_soup(xml)
    urls = get_xml_urls(soup)
    # loop through the urls
    for url in urls:
        list_of_urls.append(url)
    return list_of_urls

In [3]:

metareel_sitemap = "https://www.metareel.com/sitemaps/movie_sitemap_google.xml"
urls = get_urls(metareel_sitemap)
len(urls)

9763

In [26]:
content_list = []
for idx, url in enumerate(urls[9400:]):
    content_dict = {}
    soup = make_soup(url)
    print("fetching",9400+idx)
    try:
        movie_name = soup.find("h1","details-title").text
        content_dict['movie_name'] = movie_name
    except:
        content_dict['movie_name'] = ''
    try:
        details = soup.find("div","detail-filter pb-1")
        content_dict['type'] = details.find("span").text
        content_dict['type'] = "MOVIE"
        genre = []
        for a in details.find_all('a'):
            genre.append(a.text)
        content_dict['genre'] = genre
    except:
        content_dict['type'] = "MOVIE"
        content_dict['genre'] = []
    try:
        language_tag = a.find_next("span").find_next("span")
        content_dict['language'] = language_tag.text
    except:
        content_dict['language'] = ""
    try:
        language = a.find_next("span").find_next("span")
        content_dict['release_date'] = language.find_next("span").find_next("span").text.replace("Release Date : ","")
    except:
        content_dict['release_date'] = ""
    try:
        content_dict['imdb_rating'] = soup.find("div","movie-rating-div").text.replace("/10","")
    except:
        content_dict['imdb_rating'] = ""
    stream = []
    rent = []
    buy = []
    try:
        watch_options = soup.find("div","panel-filter-info movie-filter-info")
        try:
            stream = [tag.text for tag in watch_options.find("div","panel-filter-info-row main-panel-row stream-row").find_all('a')]
        except:
            stream = []
        try:
            rent = [tag.text for tag in watch_options.find("div","panel-filter-info-row main-panel-row rent-row").find_all('a')]
        except:
            rent = []
        try:
            buy = [tag.text for tag in watch_options.find("div","panel-filter-info-row main-panel-row buy-row").find_all('a')]
        except:
            buy = []
            
        try:
            surl = []
            watch = soup.find("div", "panel-filter-info-row main-panel-row stream-row")
            tag = watch.find("div","panel-chnl-wrp")
            for link in tag.findAll('a', href = True):
                surl.append(link['href'])
                
        except:
            surl = []
            
        try:
            rurl = []
            watch = soup.find("div", "panel-filter-info-row main-panel-row rent-row")
            tag = watch.find("div","panel-chnl-wrp")
            for link in tag.findAll('a', href = True):
                rurl.append(link['href'])
                
        except:
            rurl = []
            
        try:
            burl = []
            watch = soup.find("div", "panel-filter-info-row main-panel-row buy-row")
            tag = watch.find("div","panel-chnl-wrp")
            for link in tag.findAll('a', href = True):
                burl.append(link['href'])
                
        except:
            burl = []
            
    except:
        pass
    content_dict['stream'] = stream
    content_dict['stream url'] = surl
    content_dict['rent'] = rent
    content_dict['rent url'] = rurl
    content_dict['buy'] = buy
    content_dict['buy url'] = burl
    
    try:
        content_dict['cast'] = [tag.text for tag in soup.find("div",attrs={"name":"Cast"}).find_all('a') if tag.text]
    except:
        content_dict['cast'] = []
    try:
        content_dict['crew'] = [tag.text for tag in soup.find("div",attrs={"name":"Crew"}).find_all('a') if tag.text]
    except:
        content_dict['crew'] = []
    content_list.append(content_dict)

fetching 9400
fetching 9401
fetching 9402
fetching 9403
fetching 9404
fetching 9405
fetching 9406
fetching 9407
fetching 9408
fetching 9409
fetching 9410
fetching 9411
fetching 9412
fetching 9413
fetching 9414
fetching 9415
fetching 9416
fetching 9417
fetching 9418
fetching 9419
fetching 9420
fetching 9421
fetching 9422
fetching 9423
fetching 9424
fetching 9425
fetching 9426
fetching 9427
fetching 9428
fetching 9429
fetching 9430
fetching 9431
fetching 9432
fetching 9433
fetching 9434
fetching 9435
fetching 9436
fetching 9437
fetching 9438
fetching 9439
fetching 9440
fetching 9441
fetching 9442
fetching 9443
fetching 9444
fetching 9445
fetching 9446
fetching 9447
fetching 9448
fetching 9449
fetching 9450
fetching 9451
fetching 9452
fetching 9453
fetching 9454
fetching 9455
fetching 9456
fetching 9457
fetching 9458
fetching 9459
fetching 9460
fetching 9461
fetching 9462
fetching 9463
fetching 9464
fetching 9465
fetching 9466
fetching 9467
fetching 9468
fetching 9469
fetching 9470
fetchi

In [27]:

movies_df = pd.DataFrame.from_dict(content_list, orient='columns')
movies_df

,movie_name,type,genre,language,release_date,imdb_rating,stream,stream url,rent,rent url,buy,buy url,cast,crew
0,Kem Chho?,MOVIE,[Drama],Gujarati,2020-1-17,7.5,[Prime Video],[https://www.primevideo.com/detail/0JTDY6HOQ4W...,[],[],[],[],[Tushar Sadhu],"[Vipul Sharma, Vipul Sharma]"
1,Luv Ni Love Storys,MOVIE,"[Drama, Romance]",Gujarati,2020-1-31,7.1,[Prime Video],[https://www.primevideo.com/detail/0PDQ2J67ZT7...,[],[],[],[],"[Shraddha Dangar, Pratik Gandhi, Deeksha Joshi]",[Parth Bharat Thakkar]
2,24 Days,MOVIE,[Drama],Malayalam,2020-1-17,,[],[],[],[],[],[],[Charulatha ],[]
3,Lata Bhagwan Kare,MOVIE,[Drama],Marathi,2020-1-17,,[],[],[],[],[],[],[],[]
4,Uraan,MOVIE,"[Drama, Romance]",বাংলা,2020-1-17,,[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Respect [ 2020 ],MOVIE,"[Music, Drama]",English,2020-8-14,,[],[],[],[],[],[],"[Jennifer Hudson, Forest Whitaker, Marlon Waya...","[Harvey Mason Jr., Scott Bernstein, Aaron L. G..."
359,The Witches [ 2020 ],MOVIE,[Fantasy],English,2020-10-8,,[],[],[],[],[],[],"[Jahzir Bruno, Codie-Lei Eastick, Octavia Spen...","[Alan Silvestri, Robert Zemeckis, Robert Zemec..."
360,The Trial of the Chicago 7 [ 2020 ],MOVIE,"[Drama, History]",English,2020-10-16,,[],[],[],[],[],[],"[Sacha Baron Cohen, Eddie Redmayne, Joseph Gor...","[Steven Spielberg, Aaron Sorkin, Aaron Sorkin,..."
361,Shakthi,MOVIE,"[Action, Drama, Thriller]",Telugu,2020-3-20,6.4,[Prime Video],[https://www.primevideo.com/detail/0P57AGOK25Y...,[],[],[],[],"[Sivakarthikeyan , Kalyani Priyadarshan, Arjun...","[P. S. Mithran, Yuvan Shankar Raja, P. S. Mith..."


In [28]:
movies_df.to_csv("movies.csv", mode = 'a')